### Airport placement analysis

In this script we figure out where we should place a new airport in the Netherlands such that the most people benefit off of it, based on population.

We also make some nice maps to visualize what is happening.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth, Marker, GeoJson
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../data/airport_data/airport_code.csv')
data.head()

In [ ]:
# Drop the unnecessary columns
data.drop(['ident', 'elevation_ft', 'continent', 'iso_region', 'scheduled_service', 'gps_code', 'local_code'], axis=1, inplace=True)

dutch_data = data[data['iso_country'] == 'NL']
# Also select civilian airports, as based on inspecting https://nl.wikipedia.org/wiki/Lijst_van_vliegvelden_in_Nederland
civ_ports = ['Amsterdam Airport Schiphol', 'Eindhoven Airport', 'Groningen Airport Eelde', 
             'Maastricht Aachen Airport', 'Rotterdam The Hague Airport', 'Lelystad Airport']
dutch_data = dutch_data[dutch_data['name'].isin(civ_ports)]

# Note Groningen Airport Eelde is not in the dataset, so we will add it manually
groningen = pd.DataFrame([[2525, 'unknown_airport', 'Groningen Airport Eelde', 53.11970138549805, 6.579440116882324, 'NL', 'Groningen']], 
                         columns=dutch_data.columns)
dutch_data = pd.concat([dutch_data, groningen])
print(dutch_data.shape)
dutch_data

In [ ]:
borders = gpd.read_file('../data/airport_data/Grenzen_van_alle_Nederlandse_gemeenten_en_provincies.kml', driver='KML')
borders.head(3)

In [ ]:
dutch_data['geometry'] = gpd.points_from_xy(dutch_data['longitude_deg'], dutch_data['latitude_deg'])
dutch_data = gpd.GeoDataFrame(dutch_data, geometry='geometry')
dutch_data.crs = borders.crs
dutch_data

In [ ]:
ax = borders.plot(figsize=(10, 10), color='none')
dutch_data.plot(ax=ax, color='red', markersize=10)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

In [ ]:
# Read population data
pop_data = pd.read_excel('../data/airport_data/voorlopige-bevolkings-gegevens-20250101.xlsx', skiprows=10, sheet_name='Tabel 1')
pop_data.drop(['Gemeentecode', 'Unnamed: 2'], axis=1, inplace=True)
pop_data.rename(columns={'Unnamed: 3': 'Inwoners'}, inplace=True)
#pop_data.head()

# Check if they are all in there
print(pop_data['Gemeentenaam'].nunique())
print(borders['NAAM'].nunique())

# Check if the names are the same
print(set(pop_data['Gemeentenaam']) - set(borders['NAAM'])) # how they appear in pop_data
print(set(borders['NAAM']) - set(pop_data['Gemeentenaam']))

In [ ]:
# I'm just going to manually fix the names
pop_data_fix = pd.read_excel('../data/airport_data/voorlopige-bevolkings-gegevens-20250101-fixed.xlsx', skiprows=10, sheet_name='Tabel 1')
pop_data_fix.drop(['Gemeentecode', 'Unnamed: 2'], axis=1, inplace=True)
pop_data_fix.rename(columns={'Unnamed: 3': 'Inwoners'}, inplace=True)
print(set(pop_data_fix['Gemeentenaam']) - set(borders['NAAM']))

In [ ]:
# Add population data to border data
borders_pop = borders.merge(pop_data_fix, left_on='NAAM', right_on='Gemeentenaam')
print(borders_pop.shape)
borders_pop = borders_pop[['NAAM', 'geometry', 'Inwoners']]
borders_pop.head(3)

In [ ]:
districts = borders_pop[['NAAM', 'geometry']].set_index('NAAM')
plot_dict = borders_pop.set_index('NAAM')['Inwoners']
plot_dict.head()

In [ ]:
# Let's then define a radius within which a municipality is considered to be 'close' to an airport
r = 30 # km
# Set crs to meters, so EPSG 28992
dutch_data.to_crs(epsg=28992, inplace=True)
borders_pop.to_crs(epsg=28992, inplace=True)

# Create a buffer around the airports
r_km_buffer = dutch_data.geometry.buffer(r * 1000)

# Make a choropleth map of population within municipalities, with the airports and their buffer overlaid
m = folium.Map(location=[52.1326, 5.2913], zoom_start=8)
for idx, row in dutch_data.iterrows():
    Marker([row['latitude_deg'], row['longitude_deg']], popup=row['name']).add_to(m)

# Add choropleth layer
Choropleth(
    geo_data=districts.__geo_interface__,
    data=plot_dict,
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
).add_to(m)

# Buffer
GeoJson(r_km_buffer).add_to(m)

m

In [ ]:
# Put buffers together and say a municipality is close if more than 50% of its area is within the buffer
union = r_km_buffer.geometry.unary_union
intersections = borders_pop.geometry.intersection(union)
intersection_areas = intersections.area
total_areas = borders_pop.geometry.area
percentage_coverage = intersection_areas / total_areas
inside_borders_pop = borders_pop[percentage_coverage > 0.5]

total_pop = borders_pop['Inwoners'].sum()
inside_pop = inside_borders_pop['Inwoners'].sum()
print(f'Fraction of population within {r} km of an airport: {inside_pop / total_pop:.2f}')

In [ ]:
# Show on the map which municipalities are within the buffer
m = folium.Map(location=[52.1326, 5.2913], zoom_start=8)
for idx, row in dutch_data.iterrows():
    Marker([row['latitude_deg'], row['longitude_deg']], popup=row['name']).add_to(m)

Choropleth(
    geo_data=districts.__geo_interface__,
    data=plot_dict,
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
).add_to(m)

# Buffer
GeoJson(r_km_buffer).add_to(m)

# Add the municipalities within the buffer
centroids = inside_borders_pop.geometry.centroid.to_crs(epsg=4326)
for idx, row in inside_borders_pop.iterrows():
    Marker([centroids.loc[idx].y, centroids.loc[idx].x], popup=row['NAAM']).add_to(m)

m

In [ ]:
# Then we create a set of coordinates where a new airport could be placed
# Based on cities in corners of the country, we have:
# Vaals (50.77324555915101, 6.01131402595673)
# Cadzand (51.368992743139195, 3.4074614802218464)
# Delfzijl (53.330972145619654, 6.924773483108634)
# Den Helder (52.956204773392216, 4.760745569011207)
# So longitude should be between 3.4 and 7.0, latitude between 50.7 and 53.5
longitude = np.linspace(3.4, 7.0, 16)
latitude = np.linspace(50.7, 53.5, 16)
xx, yy = np.meshgrid(longitude, latitude)
coords = np.vstack([xx.ravel(), yy.ravel()]).T
coords = gpd.GeoDataFrame(coords, columns=['x', 'y'], geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1]))
coords.crs = 'EPSG:4326'
coords = coords.to_crs(epsg=28992)  # get geometry in meters
coords.head()

In [ ]:
# Plot potential locations on map
m = folium.Map(location=[52.1326, 5.2913], zoom_start=8)
for idx, row in coords.iterrows():
    Marker([row['y'], row['x']], popup=(row['x'], row['y'])).add_to(m)

m

In [ ]:
# A decent amount of them are outside of the Netherlands, but we'll just ignore those
# Then for each point, pretend there is an airport there and calculate the population within the buffer
def calc_pop_fraction(point, r):
    new_row = new_row = gpd.GeoDataFrame([{'name': 'new_airport', 'geometry': point}], crs=dutch_data.crs)
    dutch_data_expanded = pd.concat([dutch_data, new_row])

    r_km_buffer = dutch_data_expanded.geometry.buffer(r * 1000)
    union = r_km_buffer.unary_union
    intersections = borders_pop.geometry.intersection(union)
    intersection_areas = intersections.area
    total_areas = borders_pop.geometry.area
    percentage_coverage = intersection_areas / total_areas
    inside_borders_pop = borders_pop[percentage_coverage > 0.5]

    fraction = inside_borders_pop['Inwoners'].sum() / total_pop
    return fraction

In [ ]:
# Run for all points
best_coords = None
best_frac = 0
for point in coords.geometry:
    fraction = calc_pop_fraction(point, r)
    if fraction > best_frac:
        best_frac = fraction
        best_coords = point
print(f'Best location: {best_coords.x}, {best_coords.y}')
print(f'Fraction of population within {r} km of an airport: {best_frac:.2f}')

In [ ]:
# Plot with the new airport
m = folium.Map(location=[52.1326, 5.2913], zoom_start=8)
for idx, row in dutch_data.iterrows():
    Marker([row['latitude_deg'], row['longitude_deg']], popup=row['name']).add_to(m)

# Add new airport, find x, y by finding best_point in coords
best_x = coords[coords['geometry'] == best_coords]['x'].iloc[0]
best_y = coords[coords['geometry'] == best_coords]['y'].iloc[0]
Marker([best_y, best_x], popup='New Airport').add_to(m)

# Add choropleth layer
Choropleth(
    geo_data=districts.__geo_interface__,
    data=plot_dict,
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
).add_to(m)

# Buffer, first add new point to dutch_data
new_row = gpd.GeoDataFrame([{'name': 'new_airport', 'geometry': best_coords}], crs=dutch_data.crs)
dutch_data_expanded = pd.concat([dutch_data, new_row])
r_km_buffer = dutch_data_expanded.geometry.buffer(r * 1000)
GeoJson(r_km_buffer).add_to(m)

m.save('30km_airport_map.html')
m

So for a radius of 30 km, the new airport would have to be southeast of Utrecht (and including Utrecht itself), which seems reasonable given the large population around there that is just out of range of other airports.

In [ ]:
# Finally write a function that performs all steps for any given radius, and makes a map with the new airport
def best_airport_place(r):
    best_coords = None
    best_frac = 0
    for point in coords.geometry:
        fraction = calc_pop_fraction(point, r)
        if fraction > best_frac:
            best_frac = fraction
            best_coords = point
    print(f'Best location: {best_coords.x}, {best_coords.y}')
    print(f'Fraction of population within {r} km of an airport: {best_frac:.2f}')
    return best_coords

def plot_airport_placement(r, best_coords):
    m = folium.Map(location=[52.1326, 5.2913], zoom_start=8)
    for idx, row in dutch_data.iterrows():
        Marker([row['latitude_deg'], row['longitude_deg']], popup=row['name']).add_to(m)

    # Add new airport, find x, y by finding best_point in coords
    best_x = coords[coords['geometry'] == best_coords]['x'].iloc[0]
    best_y = coords[coords['geometry'] == best_coords]['y'].iloc[0]
    Marker([best_y, best_x], popup='New Airport').add_to(m)

    # Add choropleth layer
    Choropleth(
        geo_data=districts.__geo_interface__,
        data=plot_dict,
        key_on='feature.id',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Population'
    ).add_to(m)

    # Buffer, first add new point to dutch_data
    new_row = gpd.GeoDataFrame([{'name': 'new_airport', 'geometry': best_coords}], crs=dutch_data.crs)
    dutch_data_expanded = pd.concat([dutch_data, new_row])
    r_km_buffer = dutch_data_expanded.geometry.buffer(r * 1000)
    GeoJson(r_km_buffer).add_to(m)

    return m

In [ ]:
best_coords_10 = best_airport_place(10)
m = plot_airport_placement(10, best_coords_10)
m.save('10km_airport_map.html')
m

Given that Schiphol does not seem to cover Amsterdam sufficiently with such a small radius, it makes sense for a new airport to be placed so that Amsterdam is covered now.

In [ ]:
best_coords_50 = best_airport_place(50)
m = plot_airport_placement(50, best_coords_50)
m.save('50km_airport_map.html')
m

With such a large radius we see the final larger cities (Arnhem, Enschede, Nijmegen) and as many areas in the Netherlands as possible are covered by the new airport, as we wanted. All cities that stand out on this colourmap are serviced at this radius with the new airport.